In [343]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
%matplotlib inline

from sklearn import linear_model
from sklearn.metrics import mean_squared_error as unkind_squared_error, r2_score
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from numbers import Number


fifa_file = pd.read_csv('fifa21_train.csv')
# dropped_joined_loan_date = False
# dropped_id_name = False
# dropped_other_cols = False

###
### Exploring

In [344]:
fifa_file.shape

(11701, 101)

In [345]:
pd.set_option('max_columns',150)
fifa_file.head()

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,foot,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Contract,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,Right,1,"Jul 1, 2015",NaN,€525K,€4K,€801K,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,Right,0,"Jan 16, 2015","Dec 31, 2020",€8.5M,€23K,€0,"Dec 31, 2020 On Loan",365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77+0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,Right,0,"Jan 31, 2019",NaN,€9M,€49K,€15.3M,2019 ~ 2022,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,73,73+2,73+2,73+2,80+0,79+0,79+0,79+0,80+0,80+0,80+0,80+0,79+1,74+2,74+2,74+2,79+1,59+2,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,Right,13,"Jul 1, 2016",NaN,€275K,€4K,€694K,2016 ~ 2021,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,7,50+2,50+2,50+2,51+0,51+0,51+0,51+0,51+0,53+2,53+2,53+2,53+2,56+2,56+2,56+2,53+2,56+2,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,Right,8,"Jul 1, 2018",NaN,€725K,€2K,€1.4M,2018 ~ 2021,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,4,56+2,56+2,56+2,59+0,59+0,59+0,59+0,59+0,61+2,61+2,61+2,62+2,63+2,63+2,63+2,62+2,64+2,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


In [346]:
pd.set_option('display.max_rows', None)
display(fifa_file.describe().T)
pd.set_option('display.max_rows', 10)

,count,mean,std,min,25%,50%,75%,max
ID,11701.0,219436.649004,37480.006004,27.0,204060.0,229040.0,244176.0,259090.0
Age,11701.0,25.270490,4.957640,16.0,21.0,25.0,29.0,43.0
Growth,11701.0,5.534655,5.810903,-1.0,0.0,4.0,10.0,26.0
Attacking,11701.0,258.482181,72.267012,33.0,232.0,271.0,306.0,437.0
Crossing,11701.0,51.593795,17.872747,6.0,41.0,56.0,65.0,94.0
Finishing,11701.0,48.048116,19.399617,3.0,33.0,52.0,64.0,95.0
Heading Accuracy,11701.0,53.494060,16.956195,5.0,46.0,56.0,65.0,93.0
Short Passing,11701.0,60.436800,13.971811,8.0,56.0,63.0,69.0,94.0
Volleys,11660.0,45.067324,17.732902,4.0,32.0,47.0,59.0,90.0
Skill,11701.0,266.497222,76.835287,43.0,232.0,279.0,319.0,470.0


In [347]:
# check = clean_df[['w/f','sm','ir','a/w','d/w']]
# check.head()

# check.nunique()

# check['d/w'].unique()

# w/f values : ['4 ★', '3 ★', '2 ★', '5 ★', '1 ★']
# sm : ['2★', '4★', '3★', '1★', '5★']
# ir: ['1 ★', '2 ★', '3 ★', '5 ★', '4 ★']
# a/w: ['High', 'Medium', 'Low', nan]
# d/w: ['Medium', 'Low', 'High', nan]

#### Check nulls

In [348]:
summary = pd.concat([fifa_file.isna().sum(), round(fifa_file.isna().sum()/len(fifa_file),4)*100 ],axis=1)
summary = summary.set_axis(['Null Volumes','Null %'], axis=1)
summary
pd.set_option('display.max_rows', None)
display(summary[summary['Null Volumes']>0])
pd.set_option('display.max_rows', 20)

# drop loan_date_end
summary[summary['Null Volumes']>0].index

,Null Volumes,Null %
Club,18,0.15
Position,288,2.46
Joined,41,0.35
Loan Date End,11072,94.62
Volleys,41,0.35
Curve,41,0.35
Agility,41,0.35
Balance,41,0.35
Jumping,41,0.35
Interceptions,7,0.06


Index(['Club', 'Position', 'Joined', 'Loan Date End', 'Volleys', 'Curve',
       'Agility', 'Balance', 'Jumping', 'Interceptions', 'Positioning',
       'Vision', 'Composure', 'Sliding Tackle', 'A/W', 'D/W'],
      dtype='object')

###
### Preprocessing Functions

In [349]:
def prep_headings(my_df, columns_that_should_be_dates):
    # remove Unnamed: 0
    # my_df = my_df.drop(columns=['Unnamed: 0'])

    # drop to lower case
    my_df.columns = list(map(lambda x:x.lower(),my_df.columns))

    # replace spaces with _
    my_df.columns = list(map(lambda x:x.replace(' ','_'),my_df.columns))

    # set 'effective_to_date' to date type
    my_df[columns_that_should_be_dates] = pd.to_datetime(my_df[columns_that_should_be_dates])
    
    return my_df

def height_to_cm(height_in_feet):
    height_in_feet = height_in_feet.replace('"','') # remove the " symbol
    split_string = height_in_feet.split("'",1) # split the string on the ' symbol
    height_in_inchest = float(split_string[0]) * 12
    height_in_inchest += float(split_string[1])
    height_in_cm = round(height_in_inchest * 2.54,1) # not really needed but go metric
    return height_in_cm

def show_me_the_money(money_string):
    
    money_string = money_string[1:]
    unit = money_string[-1]
    
    if unit == '0':
        return int(money_string)
    else:
        money_string = float(money_string[:-1])
    
    if unit == 'K':
        money_string = money_string * 1000
    elif unit == 'M':
        money_string = money_string * 1000000
    elif unit == 'B':
        money_string = money_string * 1000000000
        
    return int(money_string)


def to_units(x):
    
    unit = x[-1]
    
    if unit in ['0','1','2','3','4','5','6','7','8','9']:
        return int(x)
    else:
        x = float(x[:-1])
    
    if unit == 'K':
        x = x * 1000
    elif unit == 'M':
        x = x * 1000000
    elif unit == 'B':
        x = x * 1000000000
        
    return int(x)

def score(text):
      
    if "+" in text:
        test = text.split('+',1)
        score = float(test[0])
        return score
    else:
        return text


###
### Preprocessing

#### Initial Preprocessing

In [350]:
def preprocessing(dirty_data):

    
    global dropped_joined_loan_date
    global dropped_id_name
    global dropped_other_cols
    
    dirty_data = prep_headings(dirty_data, 'joined') # not sure if this will handle joined correctly


    dirty_data['height'] = dirty_data['height'].apply(height_to_cm)

    dirty_data['weight'] = list(map(lambda x:int(x.strip('lbs')),dirty_data['weight']))  # strip lbs and convert to int

    for col in ['value','wage','release_clause']:
        dirty_data[col] = dirty_data[col].apply(show_me_the_money)

    dirty_data['hits'] = dirty_data['hits'].apply(to_units)


    wf_dic = {'4 ★' : 4, '3 ★' : 3, '2 ★' : 2, '5 ★' : 5, '1 ★' :1}
    sm_dic = {'4★' : 4, '3★' : 3, '2★' : 2, '5★' : 5, '1★' :1}
    ir_dic = {'4 ★' : 4, '3 ★' : 3, '2 ★' : 2, '5 ★' : 5, '1 ★' :1}
    aw_dic = {'High' : 3, 'Medium' : 2, 'Low' : 1}
    dw_dic = {'High' : 3, 'Medium' : 2, 'Low' : 1}

    dirty_data['w/f'] = dirty_data['w/f'].map(wf_dic)
    dirty_data['sm'] = dirty_data['sm'].map(sm_dic)
    dirty_data['ir'] = dirty_data['ir'].map(ir_dic)
    dirty_data['a/w'] = dirty_data['a/w'].map(aw_dic)
    dirty_data['d/w'] = dirty_data['d/w'].map(dw_dic)

    #dirty_data['a/w'] = pd.to_numeric(dirty_data['a/w'])
    #dirty_data['d/w'] = pd.to_numeric(dirty_data['d/w'])
    
    # dropping columns that have pretty unique values & not sure best way to handled days...consider converting to number for model testing?
    if dropped_joined_loan_date == False:
        dirty_data = dirty_data.drop(['joined','loan_date_end'],axis=1)
        dropped_joined_loan_date = True
    
    # Deciding which categoricals to keep

    # dropping columns that have unique values
    if dropped_id_name == False:
        dirty_data = dirty_data.drop(['id','name'],axis=1)       
        dropped_id_name = True

    # dropping columns with many categorical values....for now
    if dropped_other_cols == False:
        dirty_data = dirty_data.drop(['nationality','club','position','team_&_contract','contract'],axis=1)
        dropped_other_cols = True
        
        
    # handle nulls
    
    #dirty_data['club'] = dirty_data['club'].fillna("Unknown")
    #dirty_data['position'] = dirty_data['position'].fillna("Unknown")
    
    summary = pd.concat([dirty_data.isna().sum(), round(dirty_data.isna().sum()/len(clean_df),4)*100 ],axis=1)
    summary = summary.set_axis(['Null Volumes','Null %'], axis=1)
    
    # drop loan_date_end
    summary[summary['Null Volumes']>0].index
    
    #print(summary[summary['Null Volumes']>0].index)
        
    for c in summary[summary['Null Volumes']>0].index:
        dirty_data[c] = dirty_data[c].fillna(dirty_data[c].mean())
                
    #temp = dirty_data[['a/w','d/w','sm']]
    #temp.dtypes
    #summary[summary['Null Volumes']>0].index
    
    
    for_scoring = dirty_data.select_dtypes(include=object).copy() # to parse over categoricals
    # for_scoring = for_scoring.drop(['club', 'position'],axis=1) # we don't want to parse these anyway and currently have nulls which throw errors, already dropped in preprocessing 

    for col in for_scoring.columns:
         dirty_data[col] = dirty_data[col].apply(score)
            
    return dirty_data # well, this is clean-ish now

dirty_data = fifa_file.copy()
dropped_joined_loan_date = False
dropped_id_name = False
dropped_other_cols = False
dirty_data = prep_headings(dirty_data,'joined')
cleaned_data = preprocessing(dirty_data)





In [351]:
# pd.set_option('display.max_rows', None)
# display(data['hits'].value_counts())
# pd.set_option('display.max_rows', 20)

In [352]:
# pd.set_option('display.max_rows', None)
# display(data.dtypes)
# pd.set_option('display.max_rows', 20)

In [353]:
data.head()

,age,bp,height,weight,foot,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,26,CM,175.3,161,Right,1,525000,4000,801000,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,NaN,NaN,NaN,NaN,NaN,69,51,63,63,51,60,3,58.0,58.0,58.0,61.0,62.0,62.0,62.0,61.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,59.0,59.0,59.0,59.0,59.0,58.0,54.0,54.0,54.0,58.0,15.0,64
1,30,ST,182.9,159,Right,0,8500000,23000,0,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,NaN,NaN,NaN,NaN,NaN,83,75,68,82,33,71,44,77.0,77.0,77.0,77.0,77.0,77.0,77.0,77.0,76.0,76.0,76.0,76.0,68.0,68.0,68.0,76.0,57.0,53.0,53.0,53.0,57.0,53.0,48.0,48.0,48.0,53.0,18.0,77
2,33,CAM,162.6,134,Right,0,9000000,49000,15300000,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,NaN,NaN,NaN,NaN,NaN,80,77,78,86,27,56,73,73.0,73.0,73.0,80.0,79.0,79.0,79.0,80.0,80.0,80.0,80.0,79.0,74.0,74.0,74.0,79.0,59.0,56.0,56.0,56.0,59.0,53.0,41.0,41.0,41.0,53.0,12.0,80
3,22,CDM,177.8,152,Right,13,275000,4000,694000,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,NaN,NaN,NaN,NaN,NaN,57,44,54,57,57,60,7,50.0,50.0,50.0,51.0,51.0,51.0,51.0,51.0,53.0,53.0,53.0,53.0,56.0,56.0,56.0,53.0,56.0,58.0,58.0,58.0,56.0,57.0,58.0,58.0,58.0,57.0,14.0,59
4,23,CDM,180.3,150,Right,8,725000,2000,1400000,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,NaN,NaN,NaN,NaN,NaN,66,44,60,64,60,66,4,56.0,56.0,56.0,59.0,59.0,59.0,59.0,59.0,61.0,61.0,61.0,62.0,63.0,63.0,63.0,62.0,64.0,64.0,64.0,64.0,64.0,63.0,61.0,61.0,61.0,63.0,15.0,65


###
### Model the data

#####
##### process functions

###### Implemented function names:

* correlation_pair_check
* filter_numerical_features
* encode_my_shizzle
* scaling_101_for_reptiles
* choo_choo_train_model
* apply_ordinals

###### Potential functions:

* using log scales to normalize data and combining with standard scaling

In [354]:
# potential process functions & now with some actual process functions

# correlation pair check
# description:   using a numerical data frame provides correlations between them
# input:         df with 2 numerical columns
# output:        correlation_value / P
# process:       take first column and use stats.pearsonr to get P value.

def correlation_pair_check(df):
    # does df have two columns, if not return error/create exception or whatevers
    y = df.iloc[:,0]
    X = df.iloc[:,1]
    #print(round(stats.pearsonr(X, y)[0],2))
    return stats.pearsonr(X, y)[0]

# select numerical features
# description:   drop columns that have low correlation to target or very high correlation with each other
# input:         df, target, min_correlation_with_target=0.1?, max_correlation_with_other_feature=0.9?
# options:       keep_target (drop or keep target in output)
# output:        df
# process:       pull numericals, check correlation against target, check correlation of features against each other, filter based on min/max values provided, return filtered df

def filter_numerical_features(df, target, min_correlation_with_target=0.1, max_correlation_with_other_feature=0.9):
    # exclude target from features
    features = df.drop(target, axis=1)
    # split our categoricals and numericals ?? other too i guess
    numerical_features = features.select_dtypes(include=np.number)
    categorical_features = features.select_dtypes(include=object)
    other_features = features.select_dtypes(exclude=(np.number, object))
    
    # for each column in numerical features, check against target and keep if suitable
    to_keep = []
    
    for col in numerical_features.columns:
        if abs(stats.pearsonr(numerical_features[col], df[target])[0]) > min_correlation_with_target:
            to_keep.append(col)
    
    numerical_features = numerical_features[to_keep]
    
    # for each column in numerical features, check against each other exclude *redundant columns* /// highly corretable        
    i = 1
    for col in numerical_features.columns: # loop each column
        for col2 in numerical_features.iloc[:,i:].columns: # no need to check self or prior columns x.y = y.x
            i = i + 1
            if abs(stats.pearsonr(numerical_features[col], numerical_features[col2])[0]) >= max_correlation_with_other_feature:
                to_keep.remove(col2)
    
    numerical_features = numerical_features[to_keep]
    
    return pd.concat([numerical_features, categorical_features, other_features, df[target]],axis=1), to_keep


# select categorical features
# description:   drop columns that have lots of categories - perhaps later also create 'other' when suitable
# input:         df, category_limit
# output:        df
# process:       pull categoricals, check number of unique (nunique) if over limit filter them, return filtered df

# not implemented - just going to manually drop customer id and date

# encoding (for categoricals but handle in function)
# description:    applies encoding methods to categorical data depending on provided column lists for each method
# input:          df, ?target?, onehotcategoricals, labelcategoricals, labelcategoricals_datadictionary??
# output          df
# process:pull categoricals, split into ones for onehot and ones for label encoding, bring back into data

# altered: just one shot encoding - ordinal encoding handled in different function
def encode_my_shizzle(df, categoricals_for_onehot):
    
    categoricals_onehot = df[categoricals_for_onehot]
    #categoricals_ordinal = df[categoricals_for_ordinal]

    # encode and transform desired columns, dropping first category for each column
    encoder = OneHotEncoder(drop='first').fit(categoricals_onehot)
    #encoder = OneHotEncoder().fit(categoricals_onehot)
    encoded = encoder.transform(categoricals_onehot).toarray()
    
    categoricals_onehot = pd.DataFrame(encoded)
    
    # minor fix - reuse categorical names rather than dummy ones - encoder.categories_
    for col in categoricals_onehot.columns:
        if isinstance(col, (int, float)):
            #print(str(col) + ' is a number')
            categoricals_onehot = categoricals_onehot.rename(columns={col : "Faker" + str(col)})

    # drop original columns from df and replace with encoded ones
    df = df.drop(list(categoricals_for_onehot), axis=1)
    df = pd.concat([df,categoricals_onehot], axis=1)
    
    # depericated: labelEncoder
    #categoricals_ordinal = categoricals_ordinal.apply(LabelEncoder().fit_transform)
    return df

# log stuff to normalise for use with standardscaler ???

# scaling (for numericals but handle in function)
# description:    applies scaling methods to numerical data depending on provided column lists for each method
# input:          df, ?target?, minmaxscaler_numericals, standardscaler_numericals
# output:         df
# process: pull numericals, drop target, split into ones for minmax and ones for standard, apply appropriate scaling to each, bring both back into data

def scaling_101_for_reptiles(df, numericals_for_minmaxscaler=[], numericals_for_starndardscaler=[]):
    
    # min_max_scaler
    if len(numericals_for_minmaxscaler) > 0:
        minmaxscaler_numericals = df[numericals_for_minmaxscaler]    

        X = minmaxscaler_numericals
        transformer = MinMaxScaler().fit(X)
        x_normalized = transformer.transform(X)
        minmaxscaled = pd.DataFrame(x_normalized, columns=X.columns)

        # replace df columns with mm scaled columns
        df = df.drop(numericals_for_minmaxscaler, axis=1)
        df = pd.concat([df,minmaxscaled],axis=1)

    # standard_scaler
    if len(numericals_for_starndardscaler) > 0:
        standardscaler_numericals = df[numericals_for_starndardscaler]

        X = standardscaler_numericals
        Standardtransformer = StandardScaler().fit(X)
        x_standardized = Standardtransformer.transform(X)
        x_standardized = pd.DataFrame(x_standardized,columns=X.columns)

        # replace df columns with mm scaled columns
        df = df.drop(standardscaler_numericals, axis=1)
        df = pd.concat([df,x_standardized],axis=1)
    
    return df


# train model
def choo_choo_train_model(df, target, model_name, description, size_of_test=0.2, seed=42, putout=False):
    y = df[target]
    X = df.drop([target], axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size_of_test, random_state=seed)

    #create and train model
    lm = linear_model.LinearRegression()
    model = lm.fit(X_train,y_train)
    
    predictions_train = lm.predict(X_train)
    predictions_test = lm.predict(X_test)
       
    # show performance
    if putout:
        print('*** '+description+' ***')
        print('Training/Test: ' + str((1-size_of_test)*100) + '% / ' + str(size_of_test*100) +'%')
        print('Random State:  ' + str(seed))
        print('--- training data ---')
        print('R2:   ' + str(r2_score(y_train, predictions_train)))
        print()
        print('--- test data ---')
        print('R2:       ' + str(r2_score(y_test, predictions_test)))
        print('MSE:      ' + str(unkind_squared_error(y_test,predictions_test)))
        print('RMSE/MAE: ' + str(np.sqrt(unkind_squared_error(y_test,predictions_test))))

    #return [r2_score(y_train, predictions_train),r2_score(y_test, predictions_test),np.sqrt(unkind_squared_error(y_test,predictions_test))]
    return [model_name,description,size_of_test*100,seed,r2_score(y_train, predictions_train),r2_score(y_test, predictions_test),np.sqrt(unkind_squared_error(y_test,predictions_test))]


# store results


def apply_ordinals(df, cols):

    available = ['location_code','coverage']    
    for col in cols:
        if col not in available:
            print('No ordinals applied - unhandled column provided, only location_code and/or coverage currently')
            return df
    
    coverage_dic = {'Basic' : 1, 'Extended' : 2, 'Premium' : 3}
    location_code_dic = {'Rural' : 1, 'Urban' : 2, 'Suburban' : 3}
    
    for col in cols:
        if col == 'coverage':
            df[col] = df[col].map(coverage_dic)
        if col == 'location_code':
            df[col] = df[col].map(location_code_dic)
    
    return df

    